In [ ]:
import logging
from itertools import product
from datetime import datetime, timedelta

import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from searvey import usgs

logging.basicConfig(
    level=20,
    style="{",
    format="{asctime:s}; {levelname:8s}; {threadName:23s}; {name:<25s} {lineno:5d}; {message:s}",
)

logging.getLogger("urllib3").setLevel(30)
logging.getLogger("parso").setLevel(30)

logger = logging.getLogger(__name__)

## Retrieve Station Metadata

In [ ]:
usgs_stations = usgs.get_usgs_stations()
usgs_stations

In [ ]:
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
_ = countries.plot(color="lightgrey", ax=axis, zorder=-1)
_ = usgs_stations.plot(ax=axis)
_ = axis.set_xlim(usgs_stations.total_bounds[0] - 1, usgs_stations.total_bounds[2] + 1)
_ = axis.set_ylim(usgs_stations.total_bounds[1] - 1, usgs_stations.total_bounds[3] + 1)
_ = axis.set_title(f"all USGS stations")

In [ ]:
usgs_stations.columns

## Retrieve station metadata from arbitrary polygon

In [ ]:
us_northeast = shapely.geometry.box(-75, 35, -60, 40)
us_northeast

ne_stations = usgs.get_usgs_stations(region=us_northeast)
ne_stations

In [ ]:
ne_stations[ne_stations.begin_date > "2022"]

## Retrieve USGS station data

In [ ]:
ne_data = usgs.get_usgs_data(
    usgs_metadata=ne_stations,
    endtime="2020-01-30",
    period=3,
)
ne_data

In [ ]:
def drop_all_nan_coords(ds: xr.Dataset) -> xr.Dataset:
    for coord in ds.coords:
        ds = ds.isel({
            coord: np.where(
                ds.value.notnull().sum([dim for dim in ds.coords if dim != coord])
            )[0]
        })

    return ds

ds = ne_data.isel(code=ne_data.name.str.contains('Tidal')).sel(option='').squeeze().reset_coords()
ds = drop_all_nan_coords(ds)
ds

In [ ]:
fig, axes = plt.subplots(1, 1)

for st in ds.site_no:
    da = ds.sel(site_no=st).reset_coords().value.dropna('datetime').astype('float64')
    if da.size < 2:
        continue

    _ = da.plot(ax=axes, label=st.values)
    _ = axes.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    _ = axes.set_ylabel(ds.name.values)
